In [178]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [180]:
# Environment Variables
source_bucket = 'deutsche-boerse-xetra-pds'
day_diff = 3
cols = ['ISIN', 'Mnemonic', 'SecurityDesc',
        'SecurityType', 'Currency',
        'SecurityID', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
        'EndPrice', 'TradedVolume']

In [184]:
def read_source(source_bucket):
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(source_bucket)
    return bucket
def csv_to_df(bucket, day_diff):
    df_all = pd.DataFrame(columns=cols)
    arg_date_dt =datetime.today() - timedelta(days=day_diff)
    objects = [o for o in bucket.objects.all() if datetime.strptime(o.key.split('/')[0], "%Y-%m-%d").date() >= arg_date_dt.date()]
    for each in objects:
        csv_obj_init = bucket.Object(key=each.key).get().get('Body').read().decode('utf-8')
        dat = StringIO(csv_obj_init)
        df = pd.read_csv(dat, delimiter=',')
        df_all = df_all.append(df, ignore_index=True)
    return df_all

In [ ]:
df_all = csv_to_df(bucket=read_source(source_bucket), day_diff=3)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/kaleynguyen/opt/anaconda3/envs/.venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/nr/4jyzpsz56ts7c38jvfy1ktjw0000gn/T/ipykernel_19680/517554653.py", line 1, in <module>
    df_all = csv_to_df(bucket=read_source(source_bucket), day_diff=3)
  File "/var/folders/nr/4jyzpsz56ts7c38jvfy1ktjw0000gn/T/ipykernel_19680/587342521.py", line 8, in csv_to_df
    objects = [o for o in bucket.objects.all() if datetime.strptime(o.key.split('/')[0], "%Y-%m-%d").date() >= arg_date_dt.date()]
  File "/var/folders/nr/4jyzpsz56ts7c38jvfy1ktjw0000gn/T/ipykernel_19680/587342521.py", line 8, in <listcomp>
    objects = [o for o in bucket.objects.all() if datetime.strptime(o.key.split('/')[0], "%Y-%m-%d").date() >= arg_date_dt.date()]
  File "/Users/kaleynguyen/opt/anaconda3/envs/.venv/lib/python3.9/site-packages/boto3/resources/collection.py", li

In [187]:
# Get opening price per ISIN and day
df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')
# Get closing price per ISIN and day
df_all['closing_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')
# Perform aggregation per each (ISIN,Date)
df_all = df_all.groupby(['ISIN', 'Date'], as_index=False).\
    agg({
                'opening_price': 'min',
                'closing_price': 'min',
                'MinPrice': 'min',
                'MaxPrice': 'max',
                'TradedVolume': 'sum'
}).rename(
    columns = { 'opening_price': 'opening_price_eur',
                'closing_price': 'closing_price_eur',
                'MinPrice': 'minimum_price_eur',
                'MaxPrice': 'max_price_eur',
                'TradedVolume': 'daily_traded_volume'
}
)

# Compute percent change prev closing
df_all['prev_closing_price'] = df_all.sort_values('Date').groupby(['ISIN'])['closing_price_eur'].shift(1) #lag(1)

In [188]:
df_all['prev_closing_perc'] = (df_all['closing_price_eur'] - df_all['prev_closing_price'])/df_all['closing_price_eur']*100
df_all = df_all.drop(columns=['prev_closing_price'])
df_all = df_all.round(decimals=2)

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,max_price_eur,daily_traded_volume,prev_closing_perc
0,AT000000STR1,2022-03-25,36.80,36.40,36.40,36.80,119,NaN
1,AT000000STR1,2022-03-28,36.95,36.85,36.75,37.00,260,1.22
2,AT00000FACC2,2022-03-25,7.83,7.92,7.83,7.92,74,NaN
3,AT00000FACC2,2022-03-28,8.16,8.16,8.16,8.16,50,2.94
4,AT0000606306,2022-03-25,12.39,12.53,12.32,12.77,19227,NaN


In [189]:
# Save to S3
!aws s3 ls
key = 'xetra_daily_report_' + datetime.today().strftime('%Y%m%d_%H%M%S') + '.parquet'
out_buffer = BytesIO()
df_all.to_parquet(out_buffer, index=False)
bucket_target = boto3.resource('s3').Bucket('destination-xetra')
bucket_target.put_object(Body=out_buffer.getvalue(), Key=key)

2021-11-03 14:33:54 destination-snowflake
2022-03-27 11:33:16 destination-xetra
2021-12-15 13:45:47 lending-club
2022-03-25 20:37:36 source-covid-19-jobs
2021-11-03 14:33:28 source-snowflake
2021-11-02 22:08:15 xetra-target


s3.Object(bucket_name='destination-xetra', key='xetra_daily_report_20220328_141724.parquet')

In [191]:
# Read the uploaded files
for obj in bucket_target.objects.all():
    print(obj.key)
prq_obj = bucket_target.Object(key='xetra_daily_report_20220328_141724.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)
df_report

xetra_daily_report_20220327_115215.parquet
xetra_daily_report_20220328_141724.parquet


,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,max_price_eur,daily_traded_volume,prev_closing_perc
0,AT000000STR1,2022-03-25,36.80,36.40,36.40,36.80,119,NaN
1,AT000000STR1,2022-03-28,36.95,36.85,36.75,37.00,260,1.22
2,AT00000FACC2,2022-03-25,7.83,7.92,7.83,7.92,74,NaN
3,AT00000FACC2,2022-03-28,8.16,8.16,8.16,8.16,50,2.94
4,AT0000606306,2022-03-25,12.39,12.53,12.32,12.77,19227,NaN
...,...,...,...,...,...,...,...,...
6491,XS2427474023,2022-03-28,21.32,20.95,20.95,21.32,0,-3.40
6492,XS2434891219,2022-03-25,3.98,4.05,3.96,4.08,14832,NaN
6493,XS2434891219,2022-03-28,4.28,4.33,4.28,4.33,126408,6.38
6494,XS2437455608,2022-03-25,24.71,24.06,24.06,24.71,0,NaN
